In [1]:
import numpy as np
import pickle
import sys
import bite_detection_utils as bdu

from sklearn.model_selection import train_test_split
import tensorflow as tf
import my_tensorflow_utils as mtu

c:\asm\continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
root_path ='C:/ASM/DevData/eating/eating_detection/'
x_th = -0.3
var_th = 0.25
min_bite_interval = 2*16
window_size = 6*16

In [3]:
with open('C:/ASM/DevData/eating/data/steven_uva_lab_data_combined.pkl', 'rb') as file:
    lab_data = pickle.load(file)

In [4]:
windows, windows_left, windows_right, ssml, labels = bdu.get_windows_labels_for_dataset(
    lab_data, 
    x_th = x_th, 
    var_th=var_th,
    min_bite_interval = min_bite_interval,
    window_size = window_size)

print(windows.shape, windows_left.shape, windows_right.shape, ssml.shape, np.sum(labels, axis=0))

Creating windows and labels ...
x_th, var_th:  -0.3 0.25
min_bite_interval, window_size:  32 96
(18297, 96, 9) (18297, 96, 9) (18297, 96, 9) (18297, 4) [13974  3380   943]


In [5]:
#Training Parameters
learning_rate = 0.001
num_epochs = 1000
batch_size = 128
display_step = 10

#Network Parameters
num_input = 784
num_classes = 10
keep_prob_val = 0.5

In [6]:
def one_axis_conv_net(x, name):
    x_shape =x.get_shape().as_list()
    print('Inside one_axis_net: x_shape', x_shape)
    x = tf.reshape(x, shape=[-1, x.shape[1], 1, 1])
    
    with tf.name_scope(name):
        conv_1 = conv_layer(x, size_in=1, size_out=16, ksize=[1,5,1,1], strides=[1,1,1,1], padding="SAME", name='conv_1')
        maxpool_1 = maxpool_layer(conv_1, ksize=[2,1], strides=[2,1], padding="SAME", name="maxpool_1")

        conv_2 = conv_layer(maxpool_1, size_in=16, size_out=32, ksize=[1,5,1,1], strides=[1,1,1,1], padding="SAME", name='conv_2')
        maxpool_2 = maxpool_layer(conv_2, ksize=[2,1], strides=[2,1], padding="SAME", name="maxpool_2")

        flattened = tf.reshape(maxpool_2, shape=[-1, x.shape[1]//4])
        return flattened

In [7]:
def all_axes_net(x, y, keep_prob):
    x_shape = x.get_shape().as_list()
    y_shape = y.get_shape().as_list()
    print('Inside all_axes_net: x_shape, y_shape :', x_shape, y_shape)
    
    all_axes = list(range(x_shape[-1]))
    for i in range(x_shape[-1]):
        all_axis[i] = one_axis_conv_net(x[:, :, i], name="axis_"+str(i))    
        
    combo_flattened = tf.concat(all_axes, axis=-1, name='combo_flattened')
    combo_shape = combo_flattened.get_shape().as_list()    
    
    fc_1 = fc_layer(flattened, combo_shape[-1], 512, name='FC_1', relu=True)
    fcdrop_1 = tf.nn.dropout(fc_1, keep_prob, name='fcdrop_1')    
    fc_2 = fc_layer(fcdrop_1, combo_shape[-1], 256, name='FC_2', relu=True)
    fcdrop_2 = tf.nn.dropout(fc_2, keep_prob, name='fcdrop_2')    
    logits = fc_layer(fcdrop_2, 256, y_shape[-1], name='FC_3')
    
    return logits

In [ ]:
#subject_list = list(range(-1, len(lab_data)))
subject_list = [-1]

for exclude_subject in subject_list:
    print('\n\n**************************************')
    print('Excluding subject:', exclude_subject)
    print('**************************************\n')
        
    cond = ssml[:,0]!=exclude_subject
    w = windows[cond, :, :, :]
    l = ssml[cond, -1]
    l = 
    
    path = root_path+'results/window_'+str(window_size)+'_free_'+str(int(include_free_data))+'_weighted_'+str(int(weighted))+'/'
    tensorboard_logdir = path +'tensorboard/subject_'+str(exclude_subject)+'/'
    
    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, stratify=Y, test_size=0.1)
    model = train_model(X_train, Y_train, 
                              X_val, Y_val,
                              batch_size = 128,
                              epochs=epochs,
                              tensorboard_logdir=tensorboard_logdir,
                              class_weights=class_weights)    
    
    model_path = path+'models/'
    create_directory(model_path)    
    
    model.save(model_path+'subject_'+str(exclude_subject)+'.h5')    